<a href="https://colab.research.google.com/github/tronghieu2810/DEEP-LEARNING/blob/main/UDEMY/%5BMike_X_Cohen%5D_Deep_understanding/measurePerformance/DUDL_measurePerformance_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# For DL modeling
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn

# For number-crunching
import numpy as np
import scipy.stats as stats

# For dataset management
import pandas as pd
from sklearn.model_selection import train_test_split

# For data visualization
import matplotlib.pyplot as plt
from IPython import display
display.set_matplotlib_formats('svg')
import seaborn as sns

# For timing computations
import time

import copy

import sklearn.metrics as skm

# Import and process the data

In [2]:
# Import dataset
mnist_dataset = np.loadtxt(open('sample_data/mnist_train_small.csv','rb'), delimiter=',')

#Extract labels (number IDs) and remove from data
labels = mnist_dataset[:, 0]
data   = mnist_dataset[:, 1:]

# Normalize the data to a range of [0, 1]
data_norm = data / np.max(data)

# Create train/test groups using DataLoader

In [3]:
# Convert to tensor
data_tensor   = torch.tensor(data_norm).float()
labels_tensor = torch.tensor(labels).long()

#  Use scikitlearn to split the data
train_data, test_data, train_labels, test_labels = \
    train_test_split(data_tensor, labels_tensor, test_size=0.1)

# Convert into PyTorch Datasets
train_data_set = TensorDataset(train_data, train_labels)
test_data_set  = TensorDataset(test_data, test_labels)

# Translate into Dataloader objects
batch_size   = 32
train_loader = DataLoader(dataset=train_data_set, batch_size=batch_size,
                          shuffle=True, drop_last=True)
test_loader  = DataLoader(dataset=test_data_set, 
                          batch_size=test_data_set.tensors[0].shape[0])

# Create the DL model

In [4]:
def create_the_MNIST_net():
    """
    MODEL_PERFORM_MNIST_NO7 | MODEL_PERFORM_TIME
    """
    class mnist_net(nn.Module):
        def __init__(self):
            super().__init__()

            # Input layer
            self.input = nn.Linear(784, 64)

            # Hidden layer
            self.fc1 = nn.Linear(64, 32)
            self.fc2 = nn.Linear(32, 32)

            # Output layer
            self.output = nn.Linear(32, 10)

        # Forward pass
        def forward(self, x):
            x = F.relu(self.input(x))
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))

            return self.output(x)
        
    # Create the model instance
    net = mnist_net()

    # Loss Function
    loss_func = nn.CrossEntropyLoss()

    # Optimizer
    optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

    return net, loss_func, optimizer


# Create a function that trains the model

In [5]:
def train_the_model():
    """
    MODEL_PERFORM_TIME
    """
    
    # Start the timer!
    time_in_function = time.process_time()

    num_epochs = 10

    # Create a new model
    net, loss_func, optimizer = create_the_MNIST_net()

    # Initialize
    losses    = torch.zeros(num_epochs)
    train_acc = []
    test_acc  = []

    # Loop over epochs
    for epoch_i in range(num_epochs):

        # Loop over training data batches
        batch_acc  = []
        batch_loss = [] 

        for X, y in train_loader:
            # Forward pass and loss
            y_hat = net(X)
            loss  = loss_func(y_hat,y)

            # Backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Loss from this batch
            batch_loss.append(loss.item())

            # Compute accuracy
            matches = torch.argmax(y_hat, axis=1) == y       # Booleans (True/False)
            matches_numeric = matches.float()                # Convert to numbers (1/0)
            accuracy_pct = 100 * torch.mean(matches_numeric) # Average and *100
            batch_acc.append(accuracy_pct)                   # Add to list of accuracies
        # End of batch loop.

        # Get the average training accuracy of the batches
        train_acc.append(np.mean(batch_acc))

        # The average losses accross the batches
        losses[epoch_i] = np.mean(batch_loss)

        # Test accuracy
        X, y = next(iter(test_loader)) # Extract X, y from dataloader
        with torch.no_grad():
            y_hat = net(X)
        test_acc.append(100 * torch.mean((torch.argmax(y_hat, axis=1) == y).float()))

        # Report the epoch number, computation time, accuracy
        comp_time = time.process_time() - time_in_function
        print(f'Epoch {epoch_i + 1}/{num_epochs}, elapsed time: {comp_time:.2f} sec, test accuracy: {test_acc[-1]:.0f}')

    # End epochs

    return train_acc, test_acc, losses, net

# Run the model and show the results!

In [6]:
train_acc, test_acc, losses, net = train_the_model()

Epoch 1/10, elapsed time: 1.19 sec, test accuracy: 92
Epoch 2/10, elapsed time: 2.43 sec, test accuracy: 95
Epoch 3/10, elapsed time: 3.67 sec, test accuracy: 95
Epoch 4/10, elapsed time: 4.91 sec, test accuracy: 94
Epoch 5/10, elapsed time: 6.15 sec, test accuracy: 95
Epoch 6/10, elapsed time: 7.41 sec, test accuracy: 95
Epoch 7/10, elapsed time: 8.66 sec, test accuracy: 95
Epoch 8/10, elapsed time: 9.89 sec, test accuracy: 96
Epoch 9/10, elapsed time: 11.14 sec, test accuracy: 95
Epoch 10/10, elapsed time: 12.39 sec, test accuracy: 95


In [7]:
# Start the timer!
timer_outside_function = time.process_time()

for i in range(10):
    train_the_model()

total_experiment_time = time.process_time() - timer_outside_function
print(f'\n\nTotal elapsed experiment time: {total_experiment_time/60:.2f} minutes')

Epoch 1/10, elapsed time: 1.06 sec, test accuracy: 92
Epoch 2/10, elapsed time: 2.33 sec, test accuracy: 93
Epoch 3/10, elapsed time: 3.55 sec, test accuracy: 94
Epoch 4/10, elapsed time: 4.76 sec, test accuracy: 95
Epoch 5/10, elapsed time: 6.02 sec, test accuracy: 94
Epoch 6/10, elapsed time: 7.24 sec, test accuracy: 95
Epoch 7/10, elapsed time: 8.49 sec, test accuracy: 95
Epoch 8/10, elapsed time: 9.75 sec, test accuracy: 94
Epoch 9/10, elapsed time: 11.01 sec, test accuracy: 94
Epoch 10/10, elapsed time: 12.28 sec, test accuracy: 95
Epoch 1/10, elapsed time: 1.06 sec, test accuracy: 93
Epoch 2/10, elapsed time: 2.24 sec, test accuracy: 94
Epoch 3/10, elapsed time: 3.49 sec, test accuracy: 93
Epoch 4/10, elapsed time: 4.76 sec, test accuracy: 94
Epoch 5/10, elapsed time: 6.06 sec, test accuracy: 94
Epoch 6/10, elapsed time: 7.32 sec, test accuracy: 95
Epoch 7/10, elapsed time: 8.58 sec, test accuracy: 95
Epoch 8/10, elapsed time: 9.80 sec, test accuracy: 94
Epoch 9/10, elapsed time:

# Additional explorations

In [8]:
# 1) Modify the TotalExperimentTime code so that it prints minutes and seconds. For example, 500 seconds is 
#    8 minutes and 20 seconds.
# 
# 2) Modify the code inside the training function so that the display prints on only every 5th epoch.
# 